In [1]:
from google.colab import userdata

my_secret_key = userdata.get("comet_ml_api_key")

In [2]:
! pip install --upgrade comet_ml --quiet
from comet_ml import start

experiment = start(api_key=my_secret_key, project_name="lstm_mer", workspace="nikzagl")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.5 MB/s eta 0:00:00


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nikzagl/lstm-mer/f3ed692e27b2401a91e984cf4b0f0fe8



In [3]:
from google.colab import drive

drive.mount("/content/drive/")

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Mounted at /content/drive/


In [4]:
%cd "/content/drive/MyDrive/MER Project"

/content/drive/MyDrive/MER Project


In [5]:
from torch import nn
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

In [6]:
import fnmatch
import os

files = []
for root, dirnames, filenames in os.walk("MP3-Example"):
    for filename in fnmatch.filter(filenames, "*mp3"):
        files.append(os.path.join(root, filename))

In [7]:
import re

track_ids = list()
for file in files:
    track_id = re.split("\.|-", file)[-2]
    track_ids.append(track_id)

In [8]:
import pandas as pd

musicinfo_df = pd.read_csv("Music Info.csv")

In [9]:
track_ids = pd.DataFrame(track_ids).rename({0: "track_id"}, axis=1)
track_ids = track_ids.join(musicinfo_df.set_index("track_id"), on="track_id")

In [10]:
def classify_emotions(energy, valence):
    e = energy - 0.5
    v = valence - 0.5
    if v > 0 and e > 0:
        return "Joy"
    elif v < 0 and e > 0:
        return "Anger"
    elif v < 0 and e < 0:
        return "Sad"
    else:
        return "Pleasure"

In [11]:
emotions = track_ids.apply(lambda x: classify_emotions(x.energy, x.valence), axis=1)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()

In [14]:
labels = le.fit_transform(emotions)

In [15]:
tensor_df = torch.load("mfcc_tensor.pt")

In [16]:
from sklearn.model_selection import train_test_split

tensor_df_train, tensor_df_test, labels_train, labels_test = train_test_split(
    tensor_df, labels, random_state=42
)

In [17]:
train_dataset = TensorDataset(tensor_df_train, torch.tensor(labels_train))

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random

hyper_params = {"seed": 42, "batch_size": 16, "num_epochs": 25, "learning_rate": 1e-5}


# Logging hyperparamters
# experiment.log_parameters(hyper_params)
random.seed(hyper_params["seed"])
torch.manual_seed(hyper_params["seed"])

In [20]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score

# Получение весов классов
class_weights = compute_class_weight(
    "balanced", classes=np.unique(labels_train), y=labels_train
)

In [21]:
class LSTMCNNModel(nn.Module):

    def __init__(self, out_feature=4, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.conv = nn.Conv2d(1, 1, kernel_size=5)

        self.lstm = nn.LSTM(
            1168 * 16,
            self.n_hidden,
            self.n_layers,
            dropout=self.drop_prob,
            batch_first=True,
        )

        self.dropout = nn.Dropout(drop_prob)

        self.fc = nn.Linear(n_hidden, out_feature)

    def forward(self, x):
        x = x.unsqueeze(dim=1)
        # x.shape (batch, seq_len, n_features)
        x = self.conv(x)
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, channels, -1)  # (batch_size, channels, height * width)
        l_out, l_hidden = self.lstm(x)
        # out.shape (batch, seq_len, n_hidden*direction)
        out = self.dropout(l_out)

        # out.shape (batch, out_feature)
        out = self.fc(out[:, -1, :])

        # return the final output and the hidden state
        return out

In [22]:
lstm_model = LSTMCNNModel(n_hidden=512, n_layers=2).to(device)

In [23]:
test_dataset = TensorDataset(tensor_df_test, torch.tensor(labels_test))
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [24]:
from torch.utils.data import DataLoader

# Создание датасета и загрузчика данных
train_loader = DataLoader(
    train_dataset, batch_size=hyper_params["batch_size"], shuffle=True
)

# Оптимизатор
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=hyper_params["learning_rate"])
loss_function = torch.nn.CrossEntropyLoss(
    weight=torch.tensor(class_weights).float().to(device)
)
num_epochs = hyper_params["num_epochs"]
lstm_model.train()
step = 0
# Цикл обучения
for epoch in range(num_epochs):
    losses = list()
    lstm_model.train()
    for audio, labels in train_loader:
        optimizer.zero_grad()
        audio = audio.to(device)
        labels = labels.to(device)
        outputs = lstm_model(audio)
        loss = loss_function(outputs, labels)
        experiment.log_metric(name="loss", value=loss.item(), step=step)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        step += 1
    labels_pred = list()
    lstm_model.eval()
    with torch.no_grad():
        for audio, labels in test_loader:
            audio = audio.to(device)
            _, outputs = torch.max(lstm_model(audio), 1)
            labels_pred += outputs.cpu().tolist()
    f1_weighted = f1_score(labels_test, labels_pred, average="macro")
    experiment.log_metric(name="f1_score_macro", value=f1_weighted, epoch=epoch)

In [25]:
test_labels = list()
labels_pred = list()

In [26]:
lstm_model.eval()
with torch.no_grad():
    for audio, labels in test_loader:
        audio = audio.to(device)
        _, outputs = torch.max(lstm_model(audio), 1)
        labels_pred += outputs.cpu().tolist()
    experiment.log_confusion_matrix(labels_test, labels_pred, labels=le.classes_)

In [27]:
from sklearn.metrics import classification_report

In [28]:
print(classification_report(labels_test, labels_pred))

              precision    recall  f1-score   support

           0       0.60      0.68      0.64       131
           1       0.54      0.33      0.41       129
           2       0.33      0.09      0.14        35
           3       0.49      0.85      0.62        80

    accuracy                           0.54       375
   macro avg       0.49      0.49      0.45       375
weighted avg       0.53      0.54      0.51       375



In [29]:
labels_test_decoded = le.inverse_transform(labels_test)
labels_pred_decoded = le.inverse_transform(labels_pred)

In [30]:
print(classification_report(labels_test_decoded, labels_pred_decoded))

              precision    recall  f1-score   support

       Anger       0.60      0.68      0.64       131
         Joy       0.54      0.33      0.41       129
    Pleasure       0.33      0.09      0.14        35
         Sad       0.49      0.85      0.62        80

    accuracy                           0.54       375
   macro avg       0.49      0.49      0.45       375
weighted avg       0.53      0.54      0.51       375



In [31]:
from sklearn.metrics import f1_score, balanced_accuracy_score

In [32]:
f1_score(labels_test, labels_pred, average="macro")

0.45234816788715865

In [33]:
balanced_accuracy_score(labels_test, labels_pred)

np.float64(0.4871092330061796)

In [34]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : neighbouring_fuse_229
COMET INFO:     url                   : https://www.comet.com/nikzagl/lstm-mer/f3ed692e27b2401a91e984cf4b0f0fe8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     f1_score_macro [25] : (0.32904595404595405, 0.454685894973088)
COMET INFO:     loss [1953]         : (0.7655643224716187, 1.518090009689331)
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=1baYPqUckVYDwzQtWxo6VmVppV-3IbyTR
COMET INFO:   Uploads:
COMET INFO:     confusion-matrix    : 1
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET